In [12]:
import os
import numpy as np
import pandas as pd
import pymysql

db = pymysql.connect("cs540.cbxoiclfiyyp.us-west-2.rds.amazonaws.com", "masterUsername", "1234567890")
#home_folder = os.path.expanduser("~")
#data_folder = os.path.join(home_folder, "notebook_data", "basketball")
#data_filename = os.path.join(data_folder, "TeamGameLog_2014-2015.csv")
results = pd.read_sql(('SELECT * FROM project.`TeamGameLog_2014-2015`'), db)
#print(results)
ladder = pd.read_sql(('SELECT * FROM project.`2014-2015`'), db)
db.close()

In [14]:
columns = ['Date', 'VisitorTeam', 'VisitorPts', 'HomeTeam', 'HomePts']
l = int(len(results.index)/2)
index = list(range(0, l))
df = pd.DataFrame(index=index , columns=columns)

for index, row in results.iterrows():    
    if index % 2 == 0:
        if "@" in row["MATCHUP"]: 
            visitor_team, vistorPts = row["TEAM_NAME"], row["PTS"]
            if index % 2 == 0:
                home_team = results.loc[[index+1]]["TEAM_NAME"].values[0]
                homePts = results.loc[[index+1]]["PTS"].values[0]
            else:
                home_team = results.loc[[index-1]]["TEAM_NAME"].values[0]
                homePts = results.loc[[index-1]]["PTS"].values[0]
        else:
            home_team, homePts = row["TEAM_NAME"], row["PTS"]
            if index % 2 == 0:
                visitor_team = results.loc[[index+1]]["TEAM_NAME"].values[0]
                visitorPts = results.loc[[index+1]]["PTS"].values[0]
            else:
                visitor_team = results.loc[[index-1]]["TEAM_NAME"].values[0]
                visitorPts = results.loc[[index-1]]["PTS"].values[0]
            
        df.ix[index/2]['Date'] = row['GAME_DATE']
        df.ix[index/2]['VisitorTeam'] = visitor_team
        df.ix[index/2]['VisitorPts'] = visitorPts
        df.ix[index/2]['HomeTeam'] = home_team
        df.ix[index/2]['HomePts'] = homePts

df["HomeWin"] = df["VisitorPts"] < df["HomePts"]
# Our "class values"
y_true = df["HomeWin"].values
df.ix[:5]

,Date,VisitorTeam,VisitorPts,HomeTeam,HomePts,HomeWin
0,4/15/2015,Utah Jazz,91,Houston Rockets,117,True
1,4/15/2015,Denver Nuggets,126,Golden State Warriors,133,True
2,4/15/2015,Washington Wizards,108,Cleveland Cavaliers,113,True
3,4/15/2015,Charlotte Hornets,87,Toronto Raptors,92,True
4,4/15/2015,Portland Trail Blazers,87,Dallas Mavericks,114,True
5,4/15/2015,Detroit Pistons,112,New York Knicks,90,False


In [15]:
print("Home Win percentage: {0:.1f}%".format(100 * df["HomeWin"].sum() / df["HomeWin"].count()))

Home Win percentage: 54.0%


In [16]:
df["HomeLastWin"] = False
df["VisitorLastWin"] = False
# This creates two new columns, all set to False
df.ix[:5]

,Date,VisitorTeam,VisitorPts,HomeTeam,HomePts,HomeWin,HomeLastWin,VisitorLastWin
0,4/15/2015,Utah Jazz,91,Houston Rockets,117,True,False,False
1,4/15/2015,Denver Nuggets,126,Golden State Warriors,133,True,False,False
2,4/15/2015,Washington Wizards,108,Cleveland Cavaliers,113,True,False,False
3,4/15/2015,Charlotte Hornets,87,Toronto Raptors,92,True,False,False
4,4/15/2015,Portland Trail Blazers,87,Dallas Mavericks,114,True,False,False
5,4/15/2015,Detroit Pistons,112,New York Knicks,90,False,False,False


In [17]:
# Now compute the actual values for these
# Did the home and visitor teams win their last game?
from collections import defaultdict
won_last = defaultdict(int)

for index, row in df.iterrows():  # Note that this is not efficient
    home_team = row["HomeTeam"]
    visitor_team = row["VisitorTeam"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    df.ix[index] = row    
    # Set current win
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]
df.ix[20:25]

,Date,VisitorTeam,VisitorPts,HomeTeam,HomePts,HomeWin,HomeLastWin,VisitorLastWin
20,4/13/2015,Orlando Magic,93,Miami Heat,100,True,True,False
21,4/13/2015,New York Knicks,112,Atlanta Hawks,108,False,True,False
22,4/13/2015,Chicago Bulls,112,Brooklyn Nets,86,False,True,False
23,4/13/2015,Dallas Mavericks,92,Utah Jazz,109,True,False,True
24,4/13/2015,Portland Trail Blazers,92,Oklahoma City Thunder,101,True,False,False
25,4/13/2015,New Orleans Pelicans,100,Minnesota Timberwolves,88,False,True,True


In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score

# Create a dataset with just the neccessary information
X_previouswins = df[["HomeLastWin", "VisitorLastWin"]].values
clf = DecisionTreeClassifier()
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print("Using just the last result from the home and visitor teams")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using just the last result from the home and visitor teams
Accuracy: 52.4%


C:\Users\Lily\Anaconda2\envs\python3.4\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
# What about win streaks?
df["HomeWinStreak"] = 0
df["VisitorWinStreak"] = 0
# Did the home and visitor teams win their last game?
from collections import defaultdict
win_streak = defaultdict(int)

for index, row in df.iterrows():  # Note that this is not efficient
    home_team = row["HomeTeam"]
    visitor_team = row["VisitorTeam"]
    row["HomeWinStreak"] = win_streak[home_team]
    row["VisitorWinStreak"] = win_streak[visitor_team]
    df.ix[index] = row    
    # Set current win
    if row["HomeWin"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

In [20]:
clf = DecisionTreeClassifier()
X_winstreak =  df[["HomeLastWin", "VisitorLastWin", "HomeWinStreak", "VisitorWinStreak"]].values
scores = cross_val_score(clf, X_winstreak, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 53.3%


In [32]:
ladder['Rank'] = 0
for index, row in ladder.iterrows():
    ladder.set_value(index, 'Rank', index+1)
#ladder

In [22]:
# We can create a new feature -- HomeTeamRanksHigher\
df["HomeTeamRanksHigher"] = 0
for index, row in df.iterrows():
    home_team = row["HomeTeam"]
    visitor_team = row["VisitorTeam"]
    
    if home_team == "New Orleans Hornets":
        home_team = "New Orleans Pelicans"
    elif visitor_team == "New Orleans Hornets":
        visitor_team = "New Orleans Pelicans"      
    
    home_rank = ladder[ladder["TEAM_NAME"] == home_team]["Rank"].values[0]
    visitor_rank = ladder[ladder["TEAM_NAME"] == visitor_team]["Rank"].values[0]
    row["HomeTeamRanksHigher"] = int(home_rank > visitor_rank)
    df.ix[index] = row
df[:5]

,Date,VisitorTeam,VisitorPts,HomeTeam,HomePts,HomeWin,HomeLastWin,VisitorLastWin,HomeWinStreak,VisitorWinStreak,HomeTeamRanksHigher
0,4/15/2015,Utah Jazz,91,Houston Rockets,117,True,False,False,0,0,0
1,4/15/2015,Denver Nuggets,126,Golden State Warriors,133,True,False,False,0,0,1
2,4/15/2015,Washington Wizards,108,Cleveland Cavaliers,113,True,False,False,0,0,0
3,4/15/2015,Charlotte Hornets,87,Toronto Raptors,92,True,False,False,0,0,1
4,4/15/2015,Portland Trail Blazers,87,Dallas Mavericks,114,True,False,False,0,0,0


In [23]:
X_homehigher =  df[["HomeLastWin", "VisitorLastWin", "HomeTeamRanksHigher"]].values
clf = DecisionTreeClassifier()
scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 54.1%


In [24]:
from sklearn.grid_search import GridSearchCV

parameter_space = {
                   "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                   }
clf = DecisionTreeClassifier()
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_homehigher, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))

Accuracy: 54.2%


C:\Users\Lily\Anaconda2\envs\python3.4\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [25]:
# Who won the last match? We ignore home/visitor for this bit
last_match_winner = defaultdict(int)
df["HomeTeamWonLast"] = 0

for index, row in df.iterrows():
    home_team = row["HomeTeam"]
    visitor_team = row["VisitorTeam"]
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering
    # Set in the row, who won the last encounter
    row["HomeTeamWonLast"] = 1 if last_match_winner[teams] == row["HomeTeam"] else 0
    df.ix[index] = row   # -------------------------------------------------------------------
    # Who won this one?
    winner = row["HomeTeam"] if row["HomeWin"] else row["VisitorTeam"]
    last_match_winner[teams] = winner
df.ix[:5]

,Date,VisitorTeam,VisitorPts,HomeTeam,HomePts,HomeWin,HomeLastWin,VisitorLastWin,HomeWinStreak,VisitorWinStreak,HomeTeamRanksHigher,HomeTeamWonLast
0,4/15/2015,Utah Jazz,91,Houston Rockets,117,True,False,False,0,0,0,0
1,4/15/2015,Denver Nuggets,126,Golden State Warriors,133,True,False,False,0,0,1,0
2,4/15/2015,Washington Wizards,108,Cleveland Cavaliers,113,True,False,False,0,0,0,0
3,4/15/2015,Charlotte Hornets,87,Toronto Raptors,92,True,False,False,0,0,1,0
4,4/15/2015,Portland Trail Blazers,87,Dallas Mavericks,114,True,False,False,0,0,0,0
5,4/15/2015,Detroit Pistons,112,New York Knicks,90,False,False,False,0,0,1,0


In [33]:
X_home_higher =  df[["HomeTeamRanksHigher", "HomeTeamWonLast"]].values
print(X_home_higher)
clf = DecisionTreeClassifier()
scores = cross_val_score(clf, X_home_higher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

[[0 0]
 [1 0]
 [0 0]
 ..., 
 [1 1]
 [0 0]
 [1 0]]
Using whether the home team is ranked higher
Accuracy: 52.6%


In [27]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()
encoding.fit(df["HomeTeam"].values)
home_teams = encoding.transform(df["HomeTeam"].values)
visitor_teams = encoding.transform(df["VisitorTeam"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T

onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).todense()

clf = DecisionTreeClassifier()
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 56.3%


In [28]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Using full team labels is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using full team labels is ranked higher
Accuracy: 55.2%


In [29]:
X_all = np.hstack([X_home_higher, X_teams])
print(X_all.shape)

(1230, 62)


In [34]:
clf = RandomForestClassifier()
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 56.1%


In [36]:
#n_estimators=10, criterion='gini', max_depth=None, 
#min_samples_split=2, min_samples_leaf=1,
#max_features='auto',
#max_leaf_nodes=None, bootstrap=True,
#oob_score=False, n_jobs=1,
#random_state=None, verbose=0, min_density=None, compute_importances=None
parameter_space = {
                   "max_features": [2, 10, 'auto'],
                   "n_estimators": [100,],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [2, 4, 6],
                   }
clf = RandomForestClassifier()
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)

Accuracy: 60.2%
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=6,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
